# Standardizing our key columns before merging dataframes
Many a times we want to merge different data frames. But often, there is a significant challenge when our key columns of different data frames are not standardised. They contain the same elements, but there could be a spelling mistake or leading spaces etc. Let's see two such data frames. 


In [0]:
import pandas as pd

In [23]:
# Dataset-1: Hospital beds in India
beds = pd.read_csv('/content/Govt_HospitalBedsInIndia.csv')
beds.head()

,State/UT,Number of Govt Hospitals in Rural Areas,Number of Govt Beds in Rural Areas,Number of Govt Hospitals in Urban Areas,Number of Govt Beds in Urban Areas,Total Number of Govt Hospitals,Total Number of Govt Beds,Reference Period
0,Andhra Pradesh,193,6480,65,16658,258,23138,01.01.2017
1,Arunachal Pradesh*,208,2136,10,268,218,2404,31.12.2018
2,Assam *,1176,10944,50,6198,1226,17142,31.12.2017
3,Bihar,1032,5510,115,6154,1147,11664,31.12.2018
4,Chhattisgarh,169,5070,45,4342,214,9412,01.01.2016


In [21]:
# Dataset-2: Population in various states
population = pd.read_csv('/content/Population.csv')
population.head()

,State/UT,Population (2018)
0,Uttar Pradesh,"228,959,599"
1,Maharashtra,"120,837,347"
2,Bihar,"119,461,013"
3,West Bengal,"97,694,960"
4,Madhya Pradesh,"82,342,793"


You may want to merge both data sets to know per-capita availability of hospital beds in each state. But merging will not work as the state names are not standardised. 

In [24]:
percapita_beds = pd.merge(beds, population, on ='State/UT',how='inner')
percapita_beds

,State/UT,Number of Govt Hospitals in Rural Areas,Number of Govt Beds in Rural Areas,Number of Govt Hospitals in Urban Areas,Number of Govt Beds in Urban Areas,Total Number of Govt Hospitals,Total Number of Govt Beds,Reference Period,Population (2018)
0,Andhra Pradesh,193,6480,65,16658,258,23138,01.01.2017,"52,883,163"
1,Bihar,1032,5510,115,6154,1147,11664,31.12.2018,"119,461,013"
2,Chhattisgarh,169,5070,45,4342,214,9412,01.01.2016,"28,566,990"
3,Gujarat,363,11688,75,8484,438,20172,31.12.2018,"63,907,200"
4,Jammu & Kashmir,35,1221,108,6070,143,7291,31.12.2018,"13,635,010"
5,Jharkhand,519,5842,36,4942,555,10784,31.12.2015,"37,329,128"
6,Kerala,981,16865,299,21139,1280,38004,01.01.2017,"35,330,888"
7,Madhya Pradesh,330,9900,135,21206,465,31106,01.01.2018,"82,342,793"
8,Maharashtra,273,12398,438,39048,711,51446,31.12.2015,"120,837,347"
9,Manipur,23,730,7,697,30,1427,01.01.2014,"3,008,546"


Only 18 entries out of 36 could be part of the join. Because other 18 entries have minor differences within them. 

In [25]:
sorted(list(beds['State/UT']))

['A&N Island',
 'Andhra Pradesh',
 'Arunachal Pradesh*',
 'Assam *',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'D&N Haveli*',
 'Daman & Diu',
 'Delhi',
 'Goa*',
 'Gujarat',
 'Haryana*',
 'Himachal Pradesh*',
 'Jammu & Kashmir',
 'Jharkhand',
 'Karnataka*',
 'Kerala',
 'Lakshadweep',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya*',
 'Mizoram*',
 'Nagaland',
 'Odisha*',
 'Puducherry',
 'Punjab*',
 'Rajasthan *',
 'Sikkim*',
 'Tamil Nadu*',
 'Telangana*',
 'Tripura*',
 'Uttar Pradesh*',
 'Uttarakhand',
 'West Bengal']

In [27]:
sorted(list(population['State/UT']))

['A.& N.Islands',
 'Andhra Pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'D.& N.Haveli',
 'Daman & Diu',
 'Delhi',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jammu & Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Lakshadweep',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Odisha',
 'Puducherry',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Telangana',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal']

You can observe that many states/UTs are represented differently. 
"Andaman and Nicobar Islands" in one dataframe ; "A&N Island" in another. 

Such issues affect our JOINS. If it is for single use we can simply rename the fields bothering our join. But when we have to merge multiple data frames, you require a **standard key column**

In our case let's fix that the state names in population dataframe as the standard format. In any dataframes we work in future, the state names in those data frames should be the same as the standard format. Only then JOINs can be done easily. 

## LOGIC:
1. Select a state name and compare it with all the standard state names. After each comparison, there should be a metric to measure the matching between both strings.
2. We will select the standard state name that has maximum matching with the selected state. 
3. Rename the state name with standard name. 
4. Repeat with other states. 

In [0]:
#Sequence Matcher helps us get the metric that measures how two strings are matching
from difflib import SequenceMatcher

#We will write a function that gives us matching score between two strings a and b. Higher the score,better the match
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [29]:
similar('Apple',"Apple")

1.0

In [30]:
similar('Apple',"Aple")

0.8888888888888888

In [31]:
similar('Apple',"Pineapple")

0.5714285714285714

In [32]:
similar('Apple',"Mango")

0.0

In [33]:
#Fix the standard state names
standard_state_names = list(population['State/UT'])

#Empty list to store the standard names of each state in our dataframe
standard_names = []

for state in list(beds['State/UT']):
  # Empty list to store all scores
  matches = []

  #Position of selected state in the list. We need it because, we will have to rename it with stadard value later.
  statepos = list(beds['State/UT']).index(state)

  for standard in standard_state_names:
    matches.append(similar(state, standard))

  #Position of the maximum score in the list. We will use it to know what is the standard state name for every state.  
  maxpos = matches.index(max(matches))
  standard_names.append(standard_state_names[maxpos])


beds['State/UT'] = pd.DataFrame(standard_names)
beds

,State/UT,Number of Govt Hospitals in Rural Areas,Number of Govt Beds in Rural Areas,Number of Govt Hospitals in Urban Areas,Number of Govt Beds in Urban Areas,Total Number of Govt Hospitals,Total Number of Govt Beds,Reference Period
0,Andhra Pradesh,193,6480,65,16658,258,23138,01.01.2017
1,Arunachal Pradesh,208,2136,10,268,218,2404,31.12.2018
2,Assam,1176,10944,50,6198,1226,17142,31.12.2017
3,Bihar,1032,5510,115,6154,1147,11664,31.12.2018
4,Chhattisgarh,169,5070,45,4342,214,9412,01.01.2016
5,Goa,18,1397,25,1615,43,3012,31.12.2018
6,Gujarat,363,11688,75,8484,438,20172,31.12.2018
7,Haryana,609,6690,59,4550,668,11240,31.12.2016
8,Himachal Pradesh,705,5665,96,6734,801,12399,31.12.2017
9,Jammu & Kashmir,35,1221,108,6070,143,7291,31.12.2018


Let's Merge the dataframes now

In [35]:
percapita_beds = pd.merge(beds, population, on ='State/UT',how='inner')
percapita_beds

,State/UT,Number of Govt Hospitals in Rural Areas,Number of Govt Beds in Rural Areas,Number of Govt Hospitals in Urban Areas,Number of Govt Beds in Urban Areas,Total Number of Govt Hospitals,Total Number of Govt Beds,Reference Period,Population (2018)
0,Andhra Pradesh,193,6480,65,16658,258,23138,01.01.2017,"52,883,163"
1,Arunachal Pradesh,208,2136,10,268,218,2404,31.12.2018,"1,528,296"
2,Assam,1176,10944,50,6198,1226,17142,31.12.2017,"34,586,234"
3,Bihar,1032,5510,115,6154,1147,11664,31.12.2018,"119,461,013"
4,Chhattisgarh,169,5070,45,4342,214,9412,01.01.2016,"28,566,990"
5,Goa,18,1397,25,1615,43,3012,31.12.2018,"1,542,750"
6,Gujarat,363,11688,75,8484,438,20172,31.12.2018,"63,907,200"
7,Haryana,609,6690,59,4550,668,11240,31.12.2016,"27,388,008"
8,Himachal Pradesh,705,5665,96,6734,801,12399,31.12.2017,"7,316,708"
9,Jammu & Kashmir,35,1221,108,6070,143,7291,31.12.2018,"13,635,010"


# PERFECT MERGE!